<div style="text-align: center; font-size: 20px;"><b>Stats216v: Statistical Learning</b></div>

<br>
<div style="text-align: center">Stanford University</div>
<div style="text-align: center">Summer 2017</div>
<div style="text-align: center">Gyu-Ho Lee (<a href="mailto:gyuhox@gmail.com">gyuhox@gmail.com</a>)</div>

#### 5. Resampling Methods

In [2]:
LoadLibraries = function() {
    library(MASS)
    install.packages("ISLR")
    library(ISLR)
    print("Libraries have been loaded!")
}

LoadLibraries()

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


[1] "Libraries have been loaded!"


#### Validation Set Approach

In [3]:
set.seed(1)

# split set of observations into two halves
# select random subset of 196 observations
# out of original 392 observations
train = sample(392, 196)

In [6]:
attach(Auto)

# fit linear regression
lm.fit = lm(mpg~horsepower, data=Auto, subset=train)
lm.fit

The following objects are masked from Auto (pos = 3):

    acceleration, cylinders, displacement, horsepower, mpg, name,
    origin, weight, year




Call:
lm(formula = mpg ~ horsepower, data = Auto, subset = train)

Coefficients:
(Intercept)   horsepower  
    40.3404      -0.1617  


In [7]:
# calculate MSE of 196 observations in validation set
mean((mpg - predict(lm.fit, Auto))[-train]^2)

[1] 26.14142

In [8]:
# estimate test error with polynomial regressions
lm.fit2 = lm(mpg~poly(horsepower,2), data=Auto, subset=train)
mean((mpg - predict(lm.fit2, Auto))[-train]^2)

[1] 19.82259

#### Leave-One-Out Cross-Validation

In [10]:
# glm for logistic regression
# without passing 'family' argument
glm.fit = glm(mpg~horsepower, data=Auto)
coef(glm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [12]:
library(boot)
cv.err = cv.glm(Auto, glm.fit)

# cross-validation result
cv.err$delta

[1] 24.23151 24.23114

In [14]:
cv.error = rep(0, 5)
for (i in 1:5) {
    glm.fit = glm(mpg~poly(horsepower,i), data=Auto)
    cv.error[i] = cv.glm(Auto, glm.fit)$delta[1]
}

# will show sharp drop form linear to quadratic fits
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

#### k-Fold Cross-Validation

In [15]:
# 10-fold Cross-Validation
set.seed(17)
cv.error.10 = rep(0, 10)
for (i in 1:10) {
    glm.fit = glm(mpg~poly(horsepower,i), data=Auto)
    cv.error.10[i] = cv.glm(Auto, glm.fit, K=10)$delta[1]
}

cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

#### Bootstrap

In [19]:
alpha.fn = function(data, index) {
    X = data$X[index]
    Y = data$Y[index]
    return ((var(Y)-cov(X,Y))/(var(X)+var(Y)-2*cov(X,Y)))
}

alpha.fn(Portfolio, 1:100)

[1] 0.5758321

In [21]:
# randomly select 100 observations from range 1 to 100
# sample(100, 100, replace=T)

# alpha.fn(Portfolio, sample(100, 100, replace=T))
boot(Portfolio, alpha.fn, R=1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original     bias    std. error
t1* 0.5758321 0.00149669  0.08978547

In [23]:
boot.fn = function(data, index) {
    return(coef(lm(mpg~horsepower, data=data, subset=index)))
}

boot.fn(Auto, 1:392)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [24]:
set.seed(1)
boot.fn(Auto, sample(392, 392, replace=T))

(Intercept)  horsepower 
 38.7387134  -0.1481952

In [27]:
# compute standard errors of 1,000 bootstrap estimates
# boot(Auto, boot.fn, R=1000)
boot(Auto, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610 -9.962127e-03  0.86697328
t2* -0.1578447 -4.364733e-05  0.00752965